In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "am"
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s" % (source_language, target_language)

In [0]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [0]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['data.csv']))
# Dataset is now stored in a Pandas Dataframe

In [0]:
data.head()

,source_sentence,target_sentence
0,Now the famine was severe in the land.,ረሃቡ በምድሪቱ ላይ በርትቶ ነበር።
1,So when they had finished eating the grain the...,ስለሆነም ከግብፅ ያመጡትን እህል በልተው ሲጨርሱ አባታቸው “ተመልሳችሁ ሂ...
2,Then Judah said to him: “The man clearly warne...,በዚህ ጊዜ ይሁዳ እንዲህ አለው፦ “ሰውየው ‘ወንድማችሁን ይዛችሁት ካልመጣ...
3,"If you send our brother with us, we will go do...",ወንድማችንን ከእኛ ጋር የምትልከው ከሆነ ወደዚያ ወርደን እህል እንገዛልሃለን።
4,"But if you do not send him, we will not go dow...",እሱን የማትልከው ከሆነ ግን ወደዚያ አንወርድም፤ ምክንያቱም ሰውየው ‘ወን...


In [0]:
data = data.rename(columns={"English":"source_sentence", "Amharic":"target_sentence"})

In [0]:
# Do the split between dev/test/train and create parallel corpora
num_dev_patterns = 1000
num_test_patterns = 1000
df = data
# Lower case the corpora
df["source_sentence"] = df["source_sentence"].str.lower()
df["target_sentence"] = df["target_sentence"].str.lower()


devtest = df.tail(num_dev_patterns + num_test_patterns)
test = devtest.tail(num_test_patterns)
dev = devtest.head(num_dev_patterns)
stripped = df.drop(df.tail(num_dev_patterns + num_test_patterns).index)

stripped[["source_sentence"]].to_csv("train.en", index=False)
stripped[["target_sentence"]].to_csv("train.am", index=False)

dev[["source_sentence"]].to_csv("dev.en", index=False)
dev[["target_sentence"]].to_csv("dev.am", index=False)

test[["source_sentence"]].to_csv("test.en", index=False)
test[["target_sentence"]].to_csv("test.am", index=False)


In [0]:

# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 2051 (delta 25), reused 22 (delta 10), pack-reused 2006
Receiving objects: 100% (2051/2051), 2.39 MiB | 4.43 MiB/s, done.
Resolving deltas: 100% (1414/1414), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 15.9MB/s 
     |████████████████████████████████| 307kB 40.4MB/s 
     |████████████████████████████████| 51kB 25.0MB/s 
     |████████████████████████████████| 215kB 43.9MB/s 
     |████████████████████████████████| 737kB 42.2MB/s 
     |████████████████████████████████| 51kB 21.8MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=69430 sha256=e9896e30e7cae1bf92df275966028a5faaa6afb53f46963e83bd4908a9a7eb16
  Stored in directory: /tmp/pip-ephem-wheel-cache-fd6vvoos/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for p

In [0]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
! mkdir joeynmt/data/
! mkdir joeynmt/data/enam/
! export data_path=joeynmt/data/$src$tgt/
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
#! mkdir -p $data_path
! cp train.* joeynmt/data/enam/
! cp test.* joeynmt/data/enam/
! cp dev.* joeynmt/data/enam/
! cp bpe.codes.4000 $data_path
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE amharic Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/enam/vocab.txt


cp: missing destination file operand after 'bpe.codes.4000'
Try 'cp --help' for more information.
bpe.codes.4000	dev.bpe.en  sample_data  test.en       train.en
data.csv	dev.en	    test.am	 train.am      vocab.am
dev.am		drive	    test.bpe.am  train.bpe.am  vocab.en
dev.bpe.am	joeynmt     test.bpe.en  train.bpe.en
BPE amharic Sentences
በ@@ ባሕር ውስጥ ካ@@ ሉ ሕ@@ ያ@@ ዋን ፍጥ@@ ረ@@ ታ@@ ትም አንድ ሦስተ@@ ኛው ሞተ@@ ፤ ከመ@@ ርከ@@ ቦ@@ ችም አንድ ሦስተ@@ ኛው ወደ@@ መ@@ ።
ሦስተ@@ ኛው መልአክ መለከ@@ ቱን ነፋ@@ ። እንደ መብ@@ ራት ቦ@@ ግ ያለ አንድ ት@@ ልቅ ኮ@@ ከብ@@ ም ከ@@ ሰማይ ወደ@@ ቀ@@ ፤ በ@@ ወን@@ ዞ@@ ች አንድ ሦስተ@@ ኛ@@ ና በ@@ ውኃ ምን@@ ጮ@@ ች ላይ ወደ@@ ቀ@@ ።
ኮ@@ ከ@@ ቡ ጭ@@ ቁ@@ ኝ ይ@@ ባላ@@ ል። የ@@ ውኃ@@ ውም አንድ ሦስተ@@ ኛ እንደ ጭ@@ ቁ@@ ኝ መራ@@ ራ ሆነ@@ ፤ ውኃ@@ ውም መራ@@ ራ ከመ@@ ሆኑ የተነሳ ብዙ ሰዎች በ@@ ውኃ@@ ው ጠ@@ ን@@ ቅ ሞ@@ ቱ@@ ።
አራ@@ ተኛው መልአክ መለከ@@ ቱን ነፋ@@ ። የ@@ ፀሐይ አንድ ሦስተ@@ ኛ@@ ፣ የ@@ ጨረ@@ ቃ አንድ ሦስተ@@ ኛ@@ ና የ@@ ከ@@ ዋ@@ ክብ@@ ት አንድ ሦስተ@@ ኛ ተ@@ መታ@@ ፤ ይህም የሆነው የ@@ እነዚህ አካ@@ ላት አንድ ሦስተ@@ ኛው እንዲ@@ ጨ@@ ልም እንዲሁም የቀ@@ ኑ አንድ ሦስተ@@ ኛ@@ ና የ@@ ሌሊ@@ ቱ አንድ ሦስተ@@ ኛ ብርሃን እንዳ@@ ያ@@ ገኝ ነው።
እኔ

In [0]:

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.bpe.en  test.bpe.am  train.am      train.en
dev.am		dev.en	    test.bpe.en  train.bpe.am
dev.bpe.am	test.am     test.en	 train.bpe.en


In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)
name = '%s%s' % (source_language, target_language)

config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "models/{name}_transformer/12000.ckpt" # if given, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"            # Try switching from plateau to Noam scheduling
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 14 # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 400 # Decrease this for testing
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
""".format(name=name, source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [0]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2019-10-13 13:00:12,181 Hello! This is Joey-NMT.
2019-10-13 13:00:13,769 Total params: 46480384
2019-10-13 13:00:13,770 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [14]:
! cat joeynmt/models/enam_transformer/validations.txt

Steps: 400	Loss: 188883.89062	PPL: 250.56749	bleu: 0.00000	LR: 0.00027951	*
Steps: 800	Loss: 170874.82812	PPL: 147.97906	bleu: 0.00000	LR: 0.00055902	*
Steps: 1200	Loss: 145509.53125	PPL: 70.47702	bleu: 0.00000	LR: 0.00063789	*
Steps: 1600	Loss: 132005.65625	PPL: 47.48334	bleu: 0.16893	LR: 0.00055243	*
Steps: 2000	Loss: 124469.25000	PPL: 38.09122	bleu: 0.22400	LR: 0.00049411	*
Steps: 2400	Loss: 119074.07031	PPL: 32.53144	bleu: 0.28985	LR: 0.00045105	*
Steps: 2800	Loss: 114965.37500	PPL: 28.84833	bleu: 0.00000	LR: 0.00041760	*
Steps: 3200	Loss: 112569.37500	PPL: 26.89616	bleu: 0.65852	LR: 0.00039063	*
Steps: 3600	Loss: 110032.33594	PPL: 24.97287	bleu: 0.70298	LR: 0.00036828	*
Steps: 4000	Loss: 109588.02344	PPL: 24.65049	bleu: 1.14420	LR: 0.00034939	*
Steps: 4400	Loss: 109201.29688	PPL: 24.37328	bleu: 1.11332	LR: 0.00033313	*
Steps: 4800	Loss: 108250.94531	PPL: 23.70522	bleu: 1.69124	LR: 0.00031894	*


In [15]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!mkdir "$gdrive_path/models/"
!cp -r joeynmt/models/* "$gdrive_path/models/${src}${tgt}_transformer/"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-am/models/enam_transformer/best.ckpt': Function not implemented


In [16]:
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 400	Loss: 188883.89062	PPL: 250.56749	bleu: 0.00000	LR: 0.00027951	*
Steps: 800	Loss: 170874.82812	PPL: 147.97906	bleu: 0.00000	LR: 0.00055902	*
Steps: 1200	Loss: 145509.53125	PPL: 70.47702	bleu: 0.00000	LR: 0.00063789	*
Steps: 1600	Loss: 132005.65625	PPL: 47.48334	bleu: 0.16893	LR: 0.00055243	*
Steps: 2000	Loss: 124469.25000	PPL: 38.09122	bleu: 0.22400	LR: 0.00049411	*
Steps: 2400	Loss: 119074.07031	PPL: 32.53144	bleu: 0.28985	LR: 0.00045105	*
Steps: 2800	Loss: 114965.37500	PPL: 28.84833	bleu: 0.00000	LR: 0.00041760	*
Steps: 3200	Loss: 112569.37500	PPL: 26.89616	bleu: 0.65852	LR: 0.00039063	*
Steps: 3600	Loss: 110032.33594	PPL: 24.97287	bleu: 0.70298	LR: 0.00036828	*
Steps: 4000	Loss: 109588.02344	PPL: 24.65049	bleu: 1.14420	LR: 0.00034939	*
Steps: 4400	Loss: 109201.29688	PPL: 24.37328	bleu: 1.11332	LR: 0.00033313	*
Steps: 4800	Loss: 108250.94531	PPL: 23.70522	bleu: 1.69124	LR: 0.00031894	*


In [0]:
! cd joeynmt; python3 -m joeynmt test models/enam_transformer/config.yaml


2019-10-07 21:27:56,066 -  dev bleu:   1.89 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-10-07 21:31:13,308 - test bleu:   1.40 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [17]:
! cd joeynmt; python3 -m joeynmt test models/enam_transformer/config.yaml


2019-10-13 15:11:17,887 -  dev bleu:   1.89 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-10-13 15:14:35,678 - test bleu:   1.40 [Beam search decoding with beam size = 5 and alpha = 1.0]
